# <span style="color: rgb(138, 92, 245);">I. Encuesta en Aerolíneas</span>

## <span style="color: rgb(138, 92, 245);">1. Introducción</span>

Comenzamos por importar los datos referentes a este ejercicio. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Tema oscuro por defecto
px.defaults.template = "plotly_dark"

In [2]:
df_1 = pd.read_csv('data/data_practica_encuesta.csv')

In [3]:
df_1.head()

,Id,Servicio wifi a bordo,Hora de salida/llegada conveniente,Facilidad de reserva en línea,Ubicación de la puerta,Alimentos y bebidas,Embarque en línea,Comodidad del asiento,Entretenimiento a bordo,Servicio a bordo,Servicio de sala de piernas,Manejo de equipaje,Servicio de facturación,Limpieza
0,19556,5,4,3,4,3,4,3,5,5,5,5,2,5
1,90035,1,1,3,1,5,4,5,4,4,4,4,3,5
2,12360,2,0,2,4,2,2,2,2,4,1,3,2,2
3,77959,0,0,0,2,3,4,4,1,1,1,1,3,4
4,36875,2,3,4,3,4,1,2,2,2,2,2,4,4


In [4]:
df_1.shape

(25976, 14)

El presente ejercicio tiene como objetivo analizar la opinión de los usuarios de una aerolínea sobre diversos aspectos del servicio. A partir de los datos proporcionados, se evaluará la calidad de los datos, se realizará un análisis exploratorio y se aplicará un Análisis Factorial para reducir la dimensionalidad y obtener componentes latentes que representen los factores subyacentes en la percepción de los usuarios.

## <span style="color: rgb(138, 92, 245);">2. Calidad de los datos</span>

**Objetivo**: validar si los datos son consistentes, completos y adecuados para el análisis.
Incluye:

* Verificación de valores nulos o atípicos.

* Revisión de tipos de datos.

* Identificación de valores extremos o inconsistentes.

 * Decisiones de limpieza o imputación.

In [5]:
# Funcion Para Imprimir Porcetanje de Valores Nulos y Tipos de Datos
def print_data_info(df):
    print("Información del DataFrame:")
    print(df.info())
    print("\n Porcentaje de Valores nulos por columna:")
    print(df.isnull().mean() * 100)
    print("\n Cantidad de Valores Únicos por columna:")
    for column in df.columns:
        unique_count = df[column].value_counts()
        print(f"{column}: {unique_count} valores únicos")

In [12]:
# Excluimmos la primera columna que es un ID
df_1_original = df_1.copy()
df_1 = df_1.iloc[:,1:]

In [13]:
# Imprimir Información del DataFrame y Porcentaje de Valores Nulos (Excluyendo la primera columna)
print_data_info(df_1)

Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25976 entries, 0 to 25975
Data columns (total 13 columns):
 #   Column                              Non-Null Count  Dtype
---  ------                              --------------  -----
 0   Servicio wifi a bordo               25976 non-null  int64
 1   Hora de salida/llegada conveniente  25976 non-null  int64
 2   Facilidad de reserva en línea       25976 non-null  int64
 3   Ubicación de la puerta              25976 non-null  int64
 4   Alimentos y bebidas                 25976 non-null  int64
 5   Embarque en línea                   25976 non-null  int64
 6   Comodidad del asiento               25976 non-null  int64
 7   Entretenimiento a bordo             25976 non-null  int64
 8   Servicio a bordo                    25976 non-null  int64
 9   Servicio de sala de piernas         25976 non-null  int64
 10  Manejo de equipaje                  25976 non-null  int64
 11  Servicio de facturación             2597

Los resultados anteriores nos indican que toda variable es numérica por lo que se consideraron aptas para el Análisis Factorial. Además, ninguna variable contiene valores nulos, por lo cuál no es necesario imputar o eliminar ninguna variable. Tampoco se tienen variables ***dummy***, pues ninguna es de tipo binaria (tampoco unaria).

También podemos notar que todas las variables toman valores enteros del 0 al 5, es decir, existen 6 tipos de calificaciones distintas que pueden elegir los clientes. 

In [14]:
df_1.describe()

,Servicio wifi a bordo,Hora de salida/llegada conveniente,Facilidad de reserva en línea,Ubicación de la puerta,Alimentos y bebidas,Embarque en línea,Comodidad del asiento,Entretenimiento a bordo,Servicio a bordo,Servicio de sala de piernas,Manejo de equipaje,Servicio de facturación,Limpieza
count,25976.000000,25976.000000,25976.000000,25976.000000,25976.000000,25976.000000,25976.000000,25976.000000,25976.000000,25976.000000,25976.000000,25976.000000,25976.000000
mean,2.724746,3.046812,2.756775,2.977094,3.215353,3.261665,3.449222,3.357753,3.385664,3.350169,3.633238,3.314175,3.286226
std,1.335384,1.533371,1.412951,1.282133,1.331506,1.355536,1.320090,1.338299,1.282088,1.318862,1.176525,1.269332,1.319330
min,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
25%,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,3.000000,3.000000,2.000000
50%,3.000000,3.000000,3.000000,3.000000,3.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,3.000000,3.000000
75%,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,5.000000,4.000000,4.000000,4.000000,5.000000,4.000000,4.000000
max,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


Otro punto importante que debemos revisar es verificar que existen correlaciones entre las variables que tenemos.

In [15]:
# Heatmap de correlación
corr = df_1.corr()

fig_corr = px.imshow(corr, text_auto=True, aspect="auto",
                     title="Matriz de Correlación (numéricas)",
                     color_continuous_scale="Purp")
fig_corr.show()

Por otro lado, revisamos con el metodo de Z-Score si existen valores extremos que necesitemos tratar. 

In [21]:
from scipy import stats

# --- Calcular Z-score ---
z_scores = np.abs(stats.zscore(df_1))

# --- Identificar filas con algún valor extremo ---
outliers_z = (z_scores > 3)   # umbral clásico
outlier_rows = df_1[outliers_z.any(axis=1)]

print(f"Filas con valores extremos detectadas: {len(outlier_rows)}")

Filas con valores extremos detectadas: 0


En este caso, no se detectan valores extremos o atípicos que representen un problema para nuestro Análisis Factorial. 

## <span style="color: rgb(138, 92, 245);">3. Escalamiento</span>

Sabemos que, para el Análisis Factorial, necesitaremos hacer posteriormente dos tipos de pruebas: 
* Prueba de Esfericidad de Bartlett
* Prueba KMO
Ambas pruebas se basan en la matriz de correlación que graficamos anteriormente. 

Por ello, aunque las variables no tengan escalas tan distintas, es recomendable hacer un escalamiento de datos estandar (media $\mu=0$ y desviación estandar $\sigma=1$)

In [22]:
from sklearn.preprocessing import StandardScaler

In [23]:
scaler =  StandardScaler()
columns=df_1.columns
df_1 = scaler.fit_transform(df_1)
df_1 = pd.DataFrame(data=df_1,columns=columns)
df_1.head(10)

,Servicio wifi a bordo,Hora de salida/llegada conveniente,Facilidad de reserva en línea,Ubicación de la puerta,Alimentos y bebidas,Embarque en línea,Comodidad del asiento,Entretenimiento a bordo,Servicio a bordo,Servicio de sala de piernas,Manejo de equipaje,Servicio de facturación,Limpieza
0,1.703853,0.621641,0.172143,0.797831,-0.161739,0.544692,-0.340303,1.227138,1.259170,1.250974,1.161716,-1.035348,1.298998
1,-1.291598,-1.334871,0.172143,-1.542065,1.340348,0.544692,1.174774,0.479907,0.479178,0.492730,0.311739,-0.247517,1.298998
2,-0.542735,-1.987042,-0.535609,0.797831,-0.912783,-0.930768,-1.097842,-1.014556,0.479178,-1.782001,-0.538238,-1.035348,-0.974927
3,-2.040460,-1.987042,-1.951114,-0.762100,-0.161739,0.544692,0.417235,-1.761787,-1.860800,-1.782001,-2.238193,-0.247517,0.541023
4,-0.542735,-0.030530,0.879895,0.017866,0.589304,-1.668498,-1.097842,-1.014556,-1.080807,-1.023758,-1.388216,0.540315,0.541023
5,0.206128,-0.030530,0.172143,0.017866,1.340348,1.282422,-0.340303,1.227138,0.479178,-0.265514,-2.238193,-1.823179,1.298998
6,1.703853,1.273812,1.587647,1.577797,-0.161739,1.282422,1.174774,1.227138,1.259170,1.250974,1.161716,0.540315,-0.216952
7,-0.542735,-0.682700,-0.535609,-0.762100,0.589304,0.544692,1.174774,0.479907,0.479178,0.492730,0.311739,1.328146,-0.216952
8,1.703853,-0.682700,-0.535609,-0.762100,1.340348,1.282422,1.174774,1.227138,-1.080807,-1.023758,1.161716,-0.247517,1.298998
9,-0.542735,-0.682700,-0.535609,-0.762100,-0.161739,0.544692,0.417235,0.479907,0.479178,0.492730,0.311739,1.328146,0.541023


## <span style="color: rgb(138, 92, 245);">4. Prueba de Esfericidad de Bartlett</span>

La siguiente prueba propone como hipótesis nula que la matriz de correlación sea igual a la matriz identidad de su dimensión correspondiente. El proposito es rechazar la hipótesis nula con un nivel de siginificancia $\alpha=0.05$, pues esto nos diría que sí existen correlaciones entre nuestras variables. 

$$H_0: C_{n\times n}=\text{Id}_n\,\hspace{5mm}\text{vs}\hspace{5mm} H_1: C_{n\times n}\neq\text{Id}_n $$
donde $c_{ij}=\text{Corr}(X_i,X_j)=\frac{\text{Cov}(X_i,X_j)}{\sigma_{X_i}\sigma_{X_j}}$ para toda $i,j\in\{1,2,\dots,n\}$

In [24]:
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity

In [26]:
chi2,p = calculate_bartlett_sphericity(df_1)
print("Esfericidad de Bartlett")
print("Valor de Chi : ",chi2)
print("P - value : ",p)

Esfericidad de Bartlett
Valor de Chi :  133444.60062476434
P - value :  0.0


Dado que $p-\text{value}=0.0<0.05=\alpha$, entonces rechazamos la hipótesis nula, es decir, las variables están correlacionadas con un 95% de confianza. 

## <span style="color: rgb(138, 92, 245);">5. Prueba KMO</span>

# <span style="color: rgb(138, 92, 245);">II. FNA de Masa Mamaria</span>

In [10]:
df_2 = pd.read_csv('data/data_practica_cancer.csv')

# <span style="color: rgb(138, 92, 245);">III. Lenguage de Señas</span>

In [11]:
df_3 = pd.read_csv('data/data_practica_lenguaje_señas.csv')